In [106]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


## Apportionment 

In [108]:
engine = create_engine('postgresql://postgres:<use your own password>@localhost:5432/<your own database name>')

In [109]:
query_segment_matrix = "SELECT *\
                    FROM segment_matrix"
segment_matrix= pd.read_sql_query(query_segment_matrix, engine)

In [110]:
segment_matrix['apportioned'] = segment_matrix['Customer Count'] * segment_matrix['customer_vol']

Group by market segment

In [112]:
grouped_segment_matrix = segment_matrix.join(segment_matrix.groupby('Market Segment')['apportioned'].sum(),
                         on = 'Market Segment', rsuffix='_sum')

In [113]:
grouped_segment_matrix['apportioning_percentage'] = grouped_segment_matrix.apportioned/\
                                                    grouped_segment_matrix.apportioned_sum

In [114]:
grouped_segment_matrix.head()

,Product id,Tariff type,Tariff name,Market Segment,region,Payment Type,Customer Count,customer_vol,apportioned,apportioned_sum,apportioning_percentage
0,1,Single,STANDARD TARIFF,RESIDENTIAL,North,CREDIT,110374,4.55,502201.70,56795664.69,0.008842
1,2,Single,STANDARD TARIFF,RESIDENTIAL,North,DEBIT,100033,3.99,399131.67,56795664.69,0.007028
2,3,Single,STANDARD TARIFF,RESIDENTIAL,North,PRE-PAID,146390,3.98,582632.20,56795664.69,0.010258
3,4,Single,STANDARD TARIFF,RESIDENTIAL,South,CREDIT,123060,3.88,477472.80,56795664.69,0.008407
4,5,Single,STANDARD TARIFF,RESIDENTIAL,South,DEBIT,50311,4.67,234952.37,56795664.69,0.004137


Get forecast_vol

In [115]:
query_forecast_vol = "SELECT *\
                    FROM forecast_vol"
forecast_vol= pd.read_sql_query(query_forecast_vol, engine)

In [118]:
forecast_vol_market_segment= forecast_vol.merge(grouped_segment_matrix, 
                                            left_on='region',
                                            right_on = 'region',
                                            suffixes = ("_forecast", "_segment_matrix"))

Get actual volume


In [120]:
query_actual_vol = "SELECT *\
                    FROM actual_vol"
actual_vol= pd.read_sql_query(query_actual_vol, engine, parse_dates= {'dates': 'dd/mm/yy'})

Turn all Market Segment values to upper case to match forecast vol datatable

In [121]:
actual_vol['Market Segment'] = actual_vol['Market Segment'].str.upper()

Rename actual_vol's columns

In [122]:
actual_vol = actual_vol.rename({'Days':'date','Period':'period', 'Volume':'volume'},axis = 1)

Update forecasted Volume's date to be same as actual Volume

In [123]:
import datetime

In [124]:
forecast_vol_market_segment['date'] = pd.to_datetime(forecast_vol_market_segment['date'])\
                                                    .dt.strftime("%d/%m/%Y")

Take into account apportioning percentage for forecast volume

In [127]:
forecast_vol_market_segment['volume_apportioned'] = forecast_vol_market_segment.volume*forecast_vol_market_segment.apportioning_percentage

In [128]:
forecast_vol_market_segment.head()

,date,period,region,volume,Product id,Tariff type,Tariff name,Market Segment,Payment Type,Customer Count,customer_vol,apportioned,apportioned_sum,apportioning_percentage,volume_apportioned
0,01/01/2010,1,North,4573.337617,1,Single,STANDARD TARIFF,RESIDENTIAL,CREDIT,110374,4.55,502201.70,5.679566e+07,0.008842,40.438613
1,01/01/2010,1,North,4573.337617,2,Single,STANDARD TARIFF,RESIDENTIAL,DEBIT,100033,3.99,399131.67,5.679566e+07,0.007028,32.139141
2,01/01/2010,1,North,4573.337617,3,Single,STANDARD TARIFF,RESIDENTIAL,PRE-PAID,146390,3.98,582632.20,5.679566e+07,0.010258,46.915091
3,01/01/2010,1,North,4573.337617,13,Single,STANDARD TARIFF,SMALL MEDIUM ENTERPRISE,CREDIT,14061,103.30,1452501.30,3.313445e+08,0.004384,20.047954
4,01/01/2010,1,North,4573.337617,14,Single,STANDARD TARIFF,SMALL MEDIUM ENTERPRISE,DEBIT,16655,241.70,4025513.50,3.313445e+08,0.012149,55.561609


After apportioning, volume from forecast is based on customer size 

Group by date and period, sum volume for all regions



In [148]:
forecast_vol_market_segment_filter = forecast_vol_market_segment[['Market Segment', 'date', 'period','region','volume_apportioned','apportioning_percentage']]\
                                            .groupby(['date','period','Market Segment'])['volume_apportioned']\
                                            .sum().reset_index()

In [149]:
forecast_vol_market_segment_filter = forecast_vol_market_segment_filter.rename(columns={'volume_apportioned':'volume'})

Merge both actual vol and forecast vol tables for calculating Mean Absolute Percentage Error (MAPE)

In [151]:
forecast_actual_vol_merged = forecast_vol_market_segment_filter.merge(actual_vol, how = 'inner',
                                                            on =[ 'date','period','Market Segment'],suffixes=('_forecast_vol','_actual_vol'))

In [154]:
forecast_actual_vol_merged['vol_difference'] = abs((forecast_actual_vol_merged.volume_actual_vol - forecast_actual_vol_merged.volume_forecast_vol)/forecast_actual_vol_merged.volume_actual_vol)

In [ ]:
#forecast_actual_vol_merged.groupby('Market Segment')['vol_difference'].sum()

SMALL MEDIUM ENTERPRISE with lowest Mean Absolute Percentage Error from jan to june 2010

MAPE for each day, sum over all periods 

In [156]:
forecast_actual_vol_merged.groupby(['date','Market Segment'])['vol_difference'].sum().reset_index()

,date,Market Segment,vol_difference
0,01/01/2010,INDUSTRIAL CUSTOMER,11.623160
1,01/01/2010,RESIDENTIAL,11.790311
2,01/01/2010,SMALL MEDIUM ENTERPRISE,11.834755
3,01/02/2010,INDUSTRIAL CUSTOMER,14.219882
4,01/02/2010,RESIDENTIAL,12.153542
...,...,...,...
538,31/03/2010,RESIDENTIAL,12.037132
539,31/03/2010,SMALL MEDIUM ENTERPRISE,11.537388
540,31/05/2010,INDUSTRIAL CUSTOMER,12.140003
541,31/05/2010,RESIDENTIAL,11.959011


### Visualizations


In [157]:
import plotly.express as px

Mean Absolute Percentage Error (MAPE) over days

Sort by datetime, this code helped sort in chronological order  
sorting by default - months will be messed up, eg: 30th june before 31st may

In [189]:
forecast_actual_vol_merged['date'] = pd.to_datetime(forecast_actual_vol_merged.date,format="%d/%m/%Y")
forecast_actual_vol_merged = forecast_actual_vol_merged.sort_values('date')

In [ ]:
forecast_actual_vol_merged.head()

In [ ]:
df = forecast_actual_vol_merged.groupby(['date','Market Segment'])['vol_difference'].sum().reset_index()
fig = px.line(df, x='date', y="vol_difference", facet_col='Market Segment')
fig.show()

Checking what happened on June 9th in Residential

In [ ]:
forecast_actual_vol_merged[(forecast_actual_vol_merged['date']>'2010-06-08') & (forecast_actual_vol_merged['date']< '2010-06-10')]\
                    [forecast_actual_vol_merged['Market Segment'] == 'RESIDENTIAL']

At period 48, appears to be an outlier  
Could be worth investigating the cause of the outlier.

## Top 10 most profitable products by market segment

### Finding Total Costs 

### Balance cost

In [214]:
query_balance_cost = "SELECT *\
                    FROM balance_cost"
balance_cost= pd.read_sql_query(query_balance_cost, engine, parse_dates='date')

In [ ]:
balance_cost.info()

Map Balance costs to forecast_actual_vol_merged

In [218]:
cost_merged = forecast_actual_vol_merged.merge(balance_cost, on=['date','period'])
cost_merged['balancing_cost'] = cost_merged.vol_difference * cost_merged.balance_costrate

### Period Costs

In [268]:
query_other_cost = "SELECT *\
                    FROM other_cost"
other_cost= pd.read_sql_query(query_other_cost, engine, parse_dates='date')

In [269]:
cost_merged_other = cost_merged.merge(other_cost, on=['period'])
cost_merged_other['period_cost_value'] = cost_merged_other.volume_actual_vol * cost_merged_other['Period Cost']

In [ ]:
cost_merged_other.head()

## Revenue


### Standing charge revenue  
4)	Standing charge revenue: number of customers * days * (standing charge rate *(1-tariff discount percentage))

forecast_vol_market_segment.customer_count*1(day)*( tariff.standing_charge  *(1- tariff_discount.discount))

In [239]:
query_tariffs = "SELECT *\
                    FROM tariff"
tariff= pd.read_sql_query(query_tariffs, engine)

query_tariffs_disc = "SELECT *\
                    FROM tariff_discount"
tariff_discount= pd.read_sql_query(query_tariffs_disc, engine)

In [ ]:
tariff.info()
tariff_discount.info()

Joins required

forecast_vol_market_segment(joins of segment matrix, forecast and actual vol that has been appartioned) inner join  
tariff, on tariff name, region, market segment  
tariff discount, on payment type

In [352]:
revenue = forecast_vol_market_segment.merge(tariff, on=['Tariff name','Market Segment','region']).merge(tariff_discount, on='Payment Type').merge(\
                                                        actual_vol, how = 'inner', on =[ 'date','period','Market Segment'],suffixes=('_forecast_vol','_actual_vol'))

In [353]:
revenue.groupby(['Market Segment','region', 'date','period'])['standing_charge'].sum()

Market Segment           region  date        period
INDUSTRIAL CUSTOMER      East    01/01/2010  1         798.00
                                             2         798.00
                                             3         798.00
                                             4         798.00
                                             5         798.00
                                                        ...  
SMALL MEDIUM ENTERPRISE  West    31/05/2010  44          1.08
                                             45          1.08
                                             46          1.08
                                             47          1.08
                                             48          1.08
Name: standing_charge, Length: 104256, dtype: float64

In [354]:
revenue['standing_charge_a_day']= revenue['Customer Count']*(revenue['standing_charge']* (1 - revenue['tariff_discount']))

In [355]:
revenue_volume = revenue.groupby(['date','region','Market Segment','Product id','Tariff name','Payment Type'])['apportioning_percentage'].sum().reset_index().rename({'volume_apportioned':'volume_apportioned_a_day'}, axis = 1)

Actual vol for each day

In [356]:
actual_vol.head()

,date,period,Market Segment,volume
0,01/01/2010,1,SMALL MEDIUM ENTERPRISE,6531.116823
1,01/01/2010,1,RESIDENTIAL,6300.450157
2,01/01/2010,2,SMALL MEDIUM ENTERPRISE,7635.442395
3,01/01/2010,2,RESIDENTIAL,7414.109061
4,01/01/2010,3,SMALL MEDIUM ENTERPRISE,7167.867471


In [357]:
actual_vol_day = actual_vol.groupby(['date','Market Segment'])['volume'].sum().reset_index().rename({'volume':'actual_volume_a_day'}, axis= 1)

In [360]:
revenue_vol_cost = revenue_volume.merge(tariff, on=['Tariff name','Market Segment','region']).merge(tariff_discount, on='Payment Type').merge(\
                                                        actual_vol_day, how = 'inner', on =[ 'date','Market Segment'],suffixes=('_forecast_vol','_actual_vol'))

In [361]:
revenue_vol_cost['actual_apportioned_vol'] = revenue_vol_cost['apportioning_percentage']* revenue_vol_cost['actual_volume_a_day']

5)	Unit Revenue: (tariff rate *(1-tariff discount percentage)) * revenue_vol_cost.actual_apportioned_vol

In [363]:
revenue_vol_cost['unit_revenue'] = revenue_vol_cost.tariff_amount * (1-  revenue_vol_cost.tariff_discount)* revenue_vol_cost.actual_apportioned_vol

In [391]:
revenue_standing_charge_a_day = revenue[['date','region','Market Segment','Product id','Tariff name','Payment Type','standing_charge_a_day']].drop_duplicates()

In [394]:
revenues = revenue_vol_cost[['date','region','Market Segment','Product id','Tariff name','Payment Type','unit_revenue']]\
            .merge(revenue_standing_charge_a_day, on=['date','region','Market Segment','Product id','Tariff name','Payment Type'])

revenues['total_revenue'] is the total revenue for a day


In [396]:
 
revenues['total_revenue'] = revenues['unit_revenue'] + revenues['standing_charge_a_day']

In [403]:

costs = cost_merged_other[['date','period',	'Market Segment','balancing_cost',	'period_cost_value']]
costs_cleaned = costs.groupby(['date','Market Segment']).sum().reset_index().drop('period', axis = 1)



Standardise revenues date 

In [406]:
revenues['date'] = pd.to_datetime(revenues.date,format="%d/%m/%Y")
revenues = revenues.sort_values('date')

## Profits

In [408]:
profits = revenues.merge(costs_cleaned, on=['date','Market Segment'])

Unsure where to get other costs, so other costs will be taken as 0

In [410]:
profits['profit_a_day'] = profits.total_revenue - (profits.balancing_cost + profits.period_cost_value)

Table of top 10 product id in Market Segment sorted by profits

In [ ]:
profits.groupby(['Product id'])['profit_a_day'].mean().reset_index().sort_values('profit_a_day',ascending=False ).rename({'profit_a_day':'average_profit'},axis = 1).head(10)

Plot for these 10 products